<a href="https://colab.research.google.com/github/faijurrahman/Trading/blob/main/NickMccullum/002_quantitative_momentum_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [16]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

IEX_CLOUD_API_TOKEN = 'pk_fb8e90340712490d9dac75a21b629ca8'

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [19]:
stocks = pd.read_csv('https://raw.githubusercontent.com/faijurrahman/Trading/main/Data/SnP500_Company_List.csv')
stocks = stocks.drop(stocks[stocks['Symbol'] == 'DISCA'].index)
stocks = stocks.drop(stocks[stocks['Symbol'] == 'VIAC'].index)
stocks = stocks.drop(stocks[stocks['Symbol'] == 'WLTW'].index)
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [20]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2123574671820,
 'week52high': 178.58,
 'week52low': 124.17,
 'week52highSplitAdjustOnly': 179.61,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': -0.23412380355152207,
 'sharesOutstanding': 15908118000,
 'float': 0,
 'avg10Volume': 82013492,
 'avg30Volume': 84593632,
 'day200MovingAvg': 148.19,
 'day50MovingAvg': 136.68,
 'employees': 147000,
 'ttmEPS': 6.11,
 'ttmDividendRate': 0.906784243631777,
 'dividendYield': 0.006792900169539119,
 'nextDividendDate': '',
 'exDividendDate': '2022-11-04',
 'nextEarningsDate': '2023-02-02',
 'peRatio': 21.27766371571997,
 'beta': 1.3042876917890909,
 'maxChangePercent': 50.748333074895335,
 'year5ChangePercent': 2.207668222962858,
 'year2ChangePercent': 0.048712618646957306,
 'year1ChangePercent': -0.23215548089361937,
 'ytdChangePercent': 0.0273993688909413,
 'month6ChangePercent': -0.07457604088531222,
 'month3ChangePercent': -0.03669423539183503,
 'month1ChangePercent': -0.05980245216254576,
 'day

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [21]:
data['year1ChangePercent']

-0.23215548089361937

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [25]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN
CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC,F
FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM,HBAN,HII,IBM,I

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [27]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #print(data)
    for symbol in symbol_string.split(','):
        #print(symbol)
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,129.35,-0.247990,N/A
1,AOS,61.40,-0.238298,N/A
2,ABT,111.36,-0.152183,N/A
3,ABBV,152.22,0.192184,N/A
4,ABMD,381.02,0.000000,N/A
...,...,...,...,...
497,YUM,128.89,0.011655,N/A
498,ZBRA,292.99,-0.443359,N/A
499,ZBH,123.20,-0.023549,N/A
500,ZION,51.45,-0.248882,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [28]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,OXY,65.74,0.862415,N/A
1,MPC,121.01,0.659945,N/A
2,VLO,135.32,0.650442,N/A
3,HES,150.73,0.633812,N/A
4,XOM,113.22,0.622819,N/A
5,ENPH,241.80,0.608107,N/A
6,SLB,58.00,0.553960,N/A
7,HAL,42.17,0.515202,N/A
8,KSU,293.59,0.506512,N/A
9,MCK,380.84,0.501232,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [29]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000
10000


In [30]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,OXY,65.74,0.862415,2
1,MPC,121.01,0.659945,1
2,VLO,135.32,0.650442,1
3,HES,150.73,0.633812,1
4,XOM,113.22,0.622819,1
5,ENPH,241.80,0.608107,0
6,SLB,58.00,0.553960,3
7,HAL,42.17,0.515202,4
8,KSU,293.59,0.506512,0
9,MCK,380.84,0.501232,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [31]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [32]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.213147
1      0.239044
2      0.358566
3      0.876494
4      0.635458
         ...   
497    0.663347
498    0.045817
499    0.565737
500    0.211155
501    0.209163
Name: One-Year Return Percentile, Length: 502, dtype: object
0      0.358566
1      0.565737
2      0.428287
3      0.422311
4      0.304781
         ...   
497    0.581673
498     0.37251
499    0.752988
500    0.384462
501    0.113546
Name: Six-Month Return Percentile, Length: 502, dtype: object
0      0.641434
1      0.842629
2      0.374502
3      0.386454
4      0.131474
         ...   
497    0.741036
498     0.63745
499    0.482072
500    0.219124
501    0.286853
Name: Three-Month Return Percentile, Length: 502, dtype: object
0      0.478088
1      0.796813
2      0.653386
3      0.099602
4      0.290837
         ...   
497    0.396414
498    0.984064
499    0.312749
500    0.828685
501    0.501992
Name: One-Month Return Percentile, Length: 502, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,129.35,N/A,-0.247990,0.213147,0.022964,0.358566,0.200220,0.641434,0.026718,0.478088,N/A
1,AOS,61.40,N/A,-0.238298,0.239044,0.115476,0.565737,0.276492,0.842629,0.073413,0.796813,N/A
2,ABT,111.36,N/A,-0.152183,0.358566,0.048666,0.428287,0.117286,0.374502,0.052460,0.653386,N/A
3,ABBV,152.22,N/A,0.192184,0.876494,0.045842,0.422311,0.121965,0.386454,-0.036121,0.099602,N/A
4,ABMD,381.02,N/A,0.000000,0.635458,0.000000,0.304781,0.000000,0.131474,0.000000,0.290837,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,128.89,N/A,0.011655,0.663347,0.123418,0.581673,0.237857,0.741036,0.015504,0.396414,N/A
498,ZBRA,292.99,N/A,-0.443359,0.045817,0.027734,0.37251,0.199112,0.63745,0.192245,0.984064,N/A
499,ZBH,123.20,N/A,-0.023549,0.565737,0.208465,0.752988,0.149887,0.482072,0.001252,0.312749,N/A
500,ZION,51.45,N/A,-0.248882,0.211155,0.033735,0.384462,0.052224,0.219124,0.080859,0.828685,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [33]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [34]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [35]:
portfolio_input()

Enter the value of your portfolio:10000


In [36]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,129.35,1,-0.247990,0.213147,0.022964,0.358566,0.200220,0.641434,0.026718,0.478088,0.422809
1,AOS,61.40,3,-0.238298,0.239044,0.115476,0.565737,0.276492,0.842629,0.073413,0.796813,0.611056
2,ABT,111.36,1,-0.152183,0.358566,0.048666,0.428287,0.117286,0.374502,0.052460,0.653386,0.453685
3,ABBV,152.22,1,0.192184,0.876494,0.045842,0.422311,0.121965,0.386454,-0.036121,0.099602,0.446215
4,ABMD,381.02,0,0.000000,0.635458,0.000000,0.304781,0.000000,0.131474,0.000000,0.290837,0.340637
5,ACN,283.84,0,-0.237344,0.241036,0.035010,0.390438,0.120582,0.38247,-0.021429,0.141434,0.288845
6,ATVI,76.90,2,0.176275,0.864542,-0.010156,0.270916,0.046907,0.209163,0.024481,0.458167,0.450697
7,ADM,87.39,2,0.253195,0.924303,0.204123,0.743028,0.008682,0.151394,-0.056596,0.067729,0.471614
8,ADBE,344.45,0,-0.352828,0.101594,-0.107325,0.111554,0.203981,0.649402,0.037170,0.555777,0.354582
9,AAP,152.19,1,-0.315553,0.135458,-0.140147,0.081673,-0.082674,0.045817,0.061683,0.711155,0.243526


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [37]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [38]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [39]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [40]:
writer.save()